In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
import json
import pandas as pd 
import  numpy as np 
import os 
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping



ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [4]:
TRAIN_DIR =r'C:\Users\jatin\OneDrive\Desktop\pest detection project\pest_image\Train'
TEST_DIR=r'C:\Users\jatin\OneDrive\Desktop\pest detection project\pest_image\Test'

In [ ]:
def createdataframe(dir):
    image_path=[]
    labels=[]
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_path.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return image_path,labels


In [ ]:
train= pd.DataFrame()
train['image'], train['label']=createdataframe(TRAIN_DIR)

In [ ]:
 print(train)

In [ ]:
test= pd.DataFrame()
test['image'], test['label']=createdataframe(TEST_DIR)

In [ ]:
print(test)

In [ ]:
from keras.preprocessing.image import load_img
import numpy as np
from tqdm import tqdm

def extract_features(images):
    features = []
    for image_path in tqdm(images):
        try:
            # Load the image in grayscale mode
            img = load_img(image_path, color_mode='grayscale',target_size=(128,128))
            img_array = img_to_array(img)
            img_array = img_array / 255.0
            features.append(img_array)
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
    return features

# Assuming 'train' is your DataFrame with image paths
train_features = np.array(extract_features(train['image']))
test_features = np.array(extract_features(test['image']))


In [ ]:

x_train= train_features/255.0
x_test= test_features/255.0

In [ ]:
# Check the range of pixel values in training data
print(f"Min pixel value in x_train: {np.min(x_train)}")
print(f"Max pixel value in x_train: {np.max(x_train)}")

print(f"Min pixel value in x_test: {np.min(x_test)}")
print(f"Max pixel value in x_test: {np.max(x_test)}")

# Visualize one normalized image (e.g., the first image)
plt.imshow(x_train[0].reshape(128, 128), cmap='gray')  # Reshape if needed for visualization
plt.title("Normalized Image")
plt.colorbar()  # Show color scale
plt.show()

In [ ]:
le=LabelEncoder()
le.fit(train['label'])


In [ ]:
y_train=le.transform(train['label'])
y_test=le.transform(test['label'])


In [ ]:
#Check the number of images in each class directory using Python:

import os

base_dir = r'C:\Users\jatin\OneDrive\Desktop\pest detection project\pest_image\Train'
for class_dir in os.listdir(base_dir):
    class_path = os.path.join(base_dir, class_dir)
    if os.path.isdir(class_path):
        print(f"{class_dir}: {len(os.listdir(class_path))} images")

In [ ]:
# Data augmentation applied only during training
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Specify validation split (e.g., 20% for validation)
)

# Validation data only needs rescaling (no augmentation)
val_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.2)
# Apply data augmentation to the training subset
train_data = train_datagen.flow_from_directory(
    r'C:\Users\jatin\OneDrive\Desktop\pest detection project\pest_image\Train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Automatically applies augmentation to the training subset
)

# Apply no augmentation to the validation subset
val_data = val_datagen.flow_from_directory(
    r'C:\Users\jatin\OneDrive\Desktop\pest detection project\pest_image\Train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Automatically applies no augmentation to the validation subset
)

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train=to_categorical(y_train,num_classes=12)
y_test=to_categorical(y_test,num_classes=12)
#print("Original labels:", y_train)
#print("One-hot encoded labels:\n", y_test)

In [ ]:
# Build a simple CNN model
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(12, activation='softmax')  # Ensure 12 output classes
])

# Compile the model with the Adam optimizer and learning rate adjustment
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:


# Train the model
history = model.fit(
    train_data,
    epochs=30,
    validation_data=val_data,
    verbose=1,
   
)

In [ ]:
print("Model output shape:", model.output_shape)

In [ ]:
print(train_data.class_indices)

In [ ]:
test_loss, test_accuracy = model.evaluate(train_data, verbose=1)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
model.save(r"C:\Users\jatin\OneDrive\Desktop\visual studio code\AI agriculture project pest detection.keras")

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load and resize the image
def preprocess_image(image_path, target_size):
    # Load the image from the file path
    img = load_img(image_path, target_size=target_size)  # Resize image to target size
    img_array = img_to_array(img)  # Convert to numpy array
    img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Specify the image path and target size
image_path =  r"C:\Users\jatin\OneDrive\Desktop\pest detection project\new_data for prediction the model\ant(1).jpeg"

target_size = (128, 128)  # Replace with your model's input size

# Preprocess the image
test_image = preprocess_image(image_path, target_size)

# Predict using the model
predictions = model.predict(test_image)
print("Predictions:", predictions)

In [ ]:
pest_medicine_mapping = {
    "ants": "Neem oil or insecticidal soap",
    "bees": "Imidacloprid or Pyrethroid",
    "beetle": "Horticultural oil or sulfur",
    "snail": "Horticultural oil or sulfur",
    # Add more pest-medicine mappings
}

In [ ]:
print(train_data.class_indices)

In [ ]:
def predict_pest(image_path):
    # Load and preprocess the image
    img = tf.keras.utils.load_img(image_path, target_size=(128, 128))
    img_array = tf.keras.utils.img_to_array(img) / 255.0
    img_array = tf.expand_dims(img_array, 0)

    # Make a prediction
    predictions = model.predict(img_array)
    
    # Get the predicted class index
    predicted_index = predictions.argmax()
    
    # Map the predicted index to the class name
    predicted_class = {v: k for k, v in train_data.class_indices.items()}
    class_name = predicted_class[predicted_index]  # Get the class name
  
    
    # Get the recommended medicine
    medicine = pest_medicine_mapping.get(class_name, "No recommendation available")
    
    return class_name, medicine,  

In [ ]:
image_path = r"C:\Users\jatin\OneDrive\Desktop\pest detection project\pest_image\Test\snail\snail (1).jpg"
pest, medicine = predict_pest(image_path)
print(f"Pest Detected: {pest}")
print(f"Recommended Medicine: {medicine}")
print(f"Predicted Probabilities: {predictions}")